In [21]:
!pip3 install -U plotly
!pip3 install sklearn==0.23

ERROR: Could not find a version that satisfies the requirement sklearn==0.23
ERROR: No matching distribution found for sklearn==0.23


In [1]:
# Source: Alexandru Tifrea and Fanny Yang, 2021.

# Python Notebook Commands
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from copy import deepcopy
import numpy as np
import time

import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

import ipywidgets
from ipywidgets import interact, interactive, interact_manual

import sklearn
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score, GridSearchCV

from utils import generate_data, generate_additional_data, compute_population_risk, compute_empirical_risk, repeat_experiment

# Change these values if the images don't fit for your screen.
figure_width=1200
figure_height=500

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Choosing the number of CV folds - Trade-off

When choosing the number of cross-validation (CV) folds, we have to be aware of a trade-off that is illustrated in the figure below.

Let's assume that we perform K-fold CV and we use a data set $D$, with $n$ labeled samples. Let us denote by $D_T$ the $(K-1)$ splits that we use for training and by $D_V$ the split that we use for validation. The cardinals of these sets are $\frac{n(K-1)}{K}$ and $\frac{n}{K}$ respectively.

On the one hand, we want to have enough samples in the training subset $D_T$ in order a good prediction function. If $K$ is too small, then $|D_T| << |D|$ and by training on $D_T$ we obtain an estimator that is much worse than what we could have gotten, had we trained on the whole labeled set $D$ (red line below).

On the other hand, we want to have a large enough validation split, such that the validation empirical risk is a good estimate of the population risk of a prediction function. This happens if $K$ is too large (blue line below).

We assume the following notation:

- Empirical risk computed on the validation set: $\hat{R}_{V}(f):= \frac{1}{|D_V|} \sum_{(x_i, y_i)\in D_V} \left(y_i - f(x_i)\right)^2$
- Population risk: $R(f) := \mathbb{E}_{(x, y)} \left(y - f(x)\right)^2$
- Linear regression estimator trained on the whole labeled set: $\hat{f}_{full} := \text{argmin}_f \frac{1}{|D|} \sum_{(x_i, y_i)\in D} \left(y_i - f(x_i)\right)^2$
- Linear regression estimator trained on the training subset $D_T$: $\hat{f}_{CV} := \text{argmin}_f \frac{1}{|D_T|} \sum_{(x_i, y_i)\in D_T} \left(y_i - f(x_i)\right)^2$
- Ridge regression estimator trained on the training subset $D_T$ with the regularization coefficient $\lambda$ chosen using the validation subset $D_V$: $\hat{f}_{CV}^{ridge} \left(\lambda_{CV}\right)$
- Ridge regression estimator trained on the full set $D$ with the regularization coefficient $\lambda$ chosen using the population risk: $\hat{f}_{full}^{ridge} \left(\lambda_V}\right)$


In [108]:
num_runs = 5

def get_risks_for_kfold_cv(Ks):
  n = 400
  d = 100
#   noise_sigma = 10
#   n = 100
#   d = 10
  noise_sigma = 10
  snr = 1

  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=42)
  
  lin_reg_full_set = LinearRegression(fit_intercept=False)
  lin_reg_full_set.fit(X, y)
  lin_reg_full_population_risk = compute_population_risk(beta_star, lin_reg_full_set.coef_, noise_sigma, Sigma)
  
  all_ridge_coefficients = np.arange(1, 15, 1)
  pop_risks, validation_risks, best_ridge_risk_cv = [], [], []
  for K in Ks:
    lin_reg = LinearRegression(fit_intercept=False)
    train_split_size = (n // K) * (K - 1)
    lin_reg.fit(X[:train_split_size], y[:train_split_size])
    pop_risks.append(compute_population_risk(beta_star, lin_reg.coef_, noise_sigma, Sigma))
    validation_risks.append(compute_empirical_risk(lin_reg.coef_, X[train_split_size:], y[train_split_size:]))

    ridge_cv = RidgeCV(alphas=all_ridge_coefficients, fit_intercept=False, cv=K, scoring="neg_mean_squared_error").fit(X, y)
    best_ridge_risk_cv.append(-ridge_cv.best_score_)

  # Compute the best ridge estimator using the full data set.
  best_pop_risk = np.inf
  for ridge_coef in all_ridge_coefficients:
    full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False).fit(X, y)
    curr_pop_risk = compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma)
    if best_pop_risk > curr_pop_risk:
      best_pop_risk = curr_pop_risk
  full_estimator_population_risk = np.ones_like(pop_risks) * best_pop_risk

  pop_risks = np.array(pop_risks)
  validation_risks = np.array(validation_risks)
  best_ridge_risk_cv = np.array(best_ridge_risk_cv)
  full_estimator_population_risk = np.array(full_estimator_population_risk)
  return {
      "valid_vs_pop_risk": np.fabs(pop_risks - validation_risks),
      "full_vs_cv_pop_risk": np.fabs(lin_reg_full_population_risk - pop_risks),
      "cv_ridge_vs_full_ridge": np.fabs(best_ridge_risk_cv - full_estimator_population_risk),
  }

# Ks = [2, 5, 10, 50, 100]
Ks = [2, 5, 10]
aggregated_results = repeat_experiment(num_runs, get_risks_for_kfold_cv, {"Ks": Ks})

fig = go.Figure()
# Validation risk vs Population risk.
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["valid_vs_pop_risk"], 
                         name=r"$\left|\hat{R}_{V}\left(\hat{f}_{CV}\right) - R\left(\hat{f}_{CV}\right)\right|$"))
# Full set estimator vs CV estimator (pop. risk).
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["full_vs_cv_pop_risk"], 
                         name=r"$\left|R\left(\hat{f}_{full}\right) - R\left(\hat{f}_{CV}\right)\right|$"))
# Best CV ridge vs best full ridge.
fig.add_trace(go.Scatter(x=Ks, y=aggregated_results["cv_ridge_vs_full_ridge"], 
                         name=r"$\left|\hat{R}_{V}\left(\hat{f}_{CV}^{ridge}\left(\lambda_{CV}\right)\right) - R\left(\hat{f}_{full}^{ridge}\left(\lambda^*\right)\right)\right|$"))

fig.update_layout(
  height=figure_height,
  width=figure_width,
  yaxis_title="Risk difference",
  xaxis_title="Number of folds",
)
fig.show()

# Hyperparameter tuning using CV

The figures below illustrate the aforementioned trade-off for a concrete use case: tuning the ridge coefficient of a linear regression problem.

We will compare the best value of the ridge coefficient obtained using the population risk, i.e. $\lambda^*$ with the best value obtained using $K$-fold cross-validation, i.e. $\hat{\lambda}$, where we vary the number of folds $K$.

In [9]:
n = 100
d = 10
all_noise_sigmas = [0, 0.1, 0.5, 1]
snr = 1

def plot_risk_for_different_ridge_coefficients(K, noise_sigma):
  K = int(K)
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  
  all_ridge_coefficients = np.arange(0, 100, 1)
  cv_risks = []
  full_estimator_population_risk = []
  for ridge_coef in all_ridge_coefficients:
    ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
    cv_risks.append(-scores.mean())

    full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    full_ridge_reg.fit(X, y)
    full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))
  
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=cv_risks, 
                           marker_color="blue", name=f"Cross validation estimator w/ K={K}"))  
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=full_estimator_population_risk, 
                           marker_color="red", name=f"Pop. risk of full estimator"))

  if noise_sigma == 0.5:
    yaxis_range = [0.25, 0.6]
  elif noise_sigma == 1:
    yaxis_range = [1, 1.5]
  else:
    yaxis_range = [0, 0.3]
  
  best_empirical_ridge_coef = all_ridge_coefficients[np.argmin(cv_risks)]
  best_population_ridge_coef = all_ridge_coefficients[np.argmin(full_estimator_population_risk)]
  fig.add_vline(x=best_empirical_ridge_coef, line_dash="dot", line_color="blue")
  fig.add_vline(x=best_population_ridge_coef, line_dash="dot", line_color="red")
  fig.add_annotation(
      x=best_empirical_ridge_coef+2, y=yaxis_range[1]-0.03,
      text="$\huge\hat{\lambda}$",
      showarrow=False)
  fig.add_annotation(
      x=best_population_ridge_coef-2.5, y=yaxis_range[1]-0.03,
      text="$\huge \lambda^*$",
      showarrow=False)

  fig.update_layout(
    yaxis_range=yaxis_range,
    height=figure_height,
    width=figure_width,
    yaxis_title="Risk",
    xaxis_title="Ridge coefficient",
    hovermode='x'
  )
  fig.show()

interact(plot_risk_for_different_ridge_coefficients,
         K=ipywidgets.FloatSlider(value=2,
                                  min=2,
                                  max=10,
                                  step=1,
                                  readout_format='d',
                                  description='Number of folds:',
                                  style={'description_width': 'initial'},
                                  continuous_update=False),
         noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                         value=0.5,
                                         description='Noise level:',
                                         disabled=False,
                                         style={'description_width': 'initial'},
                                         continuous_update=True),);

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='Number of folds:', max=10.0…

### Impact of the number of folds

In [61]:
n = 100
d = 10
all_noise_sigmas = [0, 0.1, 0.5, 1]
snr = 1

def plot_risk_for_different_ridge_coefficients_and_Ks(noise_sigma):
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  Ks = [2, 5, 10]
  
  fig = go.Figure()
  all_ridge_coefficients = np.arange(0, 100, 1)
  full_estimator_population_risk = []
  
  for ridge_coef in all_ridge_coefficients:
    full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
    full_ridge_reg.fit(X, y)
    full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))
  full_estimator_population_risk = np.array(full_estimator_population_risk)
  best_population_risk = np.min(full_estimator_population_risk)
  fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=full_estimator_population_risk, 
                           marker_color="red", name=f"Pop. risk of full estimator"))

  for K in Ks:
    cv_risks = []
    for ridge_coef in all_ridge_coefficients:
      ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
      scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
      cv_risks.append(-scores.mean())
    
    cv_risks = np.array(cv_risks)
    best_empirical_risk = np.min(cv_risks)
    
    # Shift all lines by a constant amount for illustration purposes. Note that this transformation 
    # does not change the ranking of the ridge coefficient values.
    cv_risks = cv_risks + (best_population_risk - best_empirical_risk)
    fig.add_trace(go.Scatter(x=all_ridge_coefficients, y=cv_risks, name=f"Cross validation estimator w/ K={K}"))  

  if noise_sigma == 0.5:
    yaxis_range = [0.27, 0.33]
  elif noise_sigma == 1:
    yaxis_range = [1, 1.33]
  else:
    yaxis_range = [0, 0.3]

  fig.update_layout(
    yaxis_range=yaxis_range,
    height=figure_height,
    width=figure_width,
    yaxis_title="Risk",
    xaxis_title="Ridge coefficient",
    hovermode='x'
  )
  fig.show()

interact(plot_risk_for_different_ridge_coefficients_and_Ks,
         noise_sigma=ipywidgets.Dropdown(options=all_noise_sigmas,
                                         value=1,
                                         description='Noise level:',
                                         disabled=False,
                                         style={'description_width': 'initial'},
                                         continuous_update=True),);

interactive(children=(Dropdown(description='Noise level:', index=3, options=(0, 0.1, 0.5, 1), style=Descriptio…

We can also see that the empirical risk minimizer converges to the population minimizer, by comparing the difference between the best ridge coefficient $\lambda$ obtained through $K$-fold cross-validation, and the best value obtained using the whole labeled set for training and the population risk for hyperparameter tuning.

In [4]:
n = 100
d = 10
snr = 1
all_noise_sigmas = [0, 0.1, 0.5, 1]
all_Ks = np.arange(2, 11)

deviation_from_best_lambdas = {}
all_ridge_coefficients = np.arange(0, 100, 1)

for noise_sigma in all_noise_sigmas:
  X, y, beta_star, Sigma = generate_data(n=n, d=d, snr=snr, noise_sigma=noise_sigma, seed=21)
  deviation_from_best_lambdas[noise_sigma] = []
  for K in all_Ks:
    cv_risks, full_estimator_population_risk = [], []
    for ridge_coef in all_ridge_coefficients:
      ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False)
      scores = cross_val_score(ridge_reg, X, y, scoring="neg_mean_squared_error", cv=K)
      cv_risks.append(-scores.mean())

      full_ridge_reg = Ridge(alpha=ridge_coef, fit_intercept=False).fit(X, y)
      full_estimator_population_risk.append(compute_population_risk(beta_star, full_ridge_reg.coef_, noise_sigma, Sigma))

    best_empirical_lambda = all_ridge_coefficients[np.argmin(cv_risks)]
    best_population_lambda = all_ridge_coefficients[np.argmin(full_estimator_population_risk)]
    deviation_from_best_lambdas[noise_sigma].append(np.fabs(best_empirical_lambda - best_population_lambda))

fig = go.Figure()
for noise_sigma in all_noise_sigmas:
  fig.add_trace(go.Scatter(x=all_Ks, y=deviation_from_best_lambdas[noise_sigma], name=f"Noise level {noise_sigma}"))

fig.update_layout(
  height=figure_height,
  width=figure_width,
  yaxis_title=r"$\text{Deviation from }\lambda^*$",
  xaxis_title="Number of folds",
  hovermode='x'
)
fig.show()